In [1]:
!pip install torchmetrics[audio]
!pip install pesq
import soundfile as sf
import torch
import torchmetrics
from torchmetrics.functional.audio.pesq import perceptual_evaluation_speech_quality
from torchmetrics.functional.audio import signal_distortion_ratio, scale_invariant_signal_distortion_ratio, signal_noise_ratio

  Created wheel for pystoi: filename=pystoi-0.3.3-py2.py3-none-any.whl size=7778 sha256=7f9227e7373db8058a08be0a2b62be4cafd6687ea00bee371346e7239fa19511
  Stored in directory: /root/.cache/pip/wheels/3b/ca/9e/5b5d6e5e109322303b50d21918ad2bd7d50a2a0775c11e08e8
Successfully built pystoi
  Preparing metadata (setup.py) ... done
  Created wheel for pesq: filename=pesq-0.0.4-cp310-cp310-linux_x86_64.whl size=262927 sha256=90067076190da5ad6b6327538c8238222c50eae8b46b018ac1b419af0108b9df
  Stored in directory: /root/.cache/pip/wheels/c5/4e/2c/251524370c0fdd659e99639a0fbd0ca5a782c3aafcd456b28d
Successfully built pesq


In [4]:
data_folder = '/content/drive/MyDrive/DSP/'

In [5]:
orig, sr = sf.read(data_folder + 'gt.wav')
orig_recorded, sr = sf.read(data_folder + 'enhanced/gt_recorded.wav')
orig_convolved, sr = sf.read(data_folder + 'enhanced/gt_convolved.wav')

In [6]:
orig = torch.tensor(orig)
orig_recorded = torch.tensor(orig_recorded[:orig.shape[0]])
orig_convolved = torch.tensor(orig_convolved)

In [7]:
def count_metrics(bad, orig):
    """
    Считает торчовые метрики
    """
    metrics = {}
    metrics['snr'] = signal_noise_ratio(bad, orig).item()
    metrics['sdr'] = signal_distortion_ratio(bad, orig).item()
    metrics['sisdr'] = scale_invariant_signal_distortion_ratio(bad, orig).item()
    metrics['pesq'] = perceptual_evaluation_speech_quality(bad, orig, fs=8000, mode='nb').item()

    return metrics

In [8]:
count_metrics(orig_recorded, orig)

{'snr': -0.2088848895814074,
 'sdr': -3.208841149288922,
 'sisdr': -11.047636021130689,
 'pesq': 2.16459584236145}

In [9]:
count_metrics(orig_convolved, orig)

{'snr': -0.2482051688654703,
 'sdr': -22.02366951755776,
 'sisdr': -53.252120880457916,
 'pesq': 4.011621952056885}

Теперь посчитаем более сложные метрики, основанные на нейронках

# NISQA

In [10]:
!git clone https://github.com/gabrielmittag/NISQA.git

Cloning into 'NISQA'...
remote: Enumerating objects: 253, done.
remote: Counting objects: 100% (253/253), done.
remote: Compressing objects: 100% (132/132), done.
remote: Total 253 (delta 127), reused 235 (delta 120), pack-reused 0
Receiving objects: 100% (253/253), 2.28 MiB | 16.53 MiB/s, done.
Resolving deltas: 100% (127/127), done.


In [14]:
!mkdir results

In [15]:
!python NISQA/run_predict.py --mode predict_file --pretrained_model NISQA/weights/nisqa.tar --deg /content/drive/MyDrive/DSP/gt.wav --output_dir ./results

Device: cpu
Model architecture: NISQA_DIM
Loaded pretrained model from NISQA/weights/nisqa.tar
---> Predicting ...
   deg  mos_pred  noi_pred  dis_pred  col_pred  loud_pred   model
gt.wav  4.706658  4.499088  4.600805  4.408635    4.54724 NISQAv2


In [16]:
!python NISQA/run_predict.py --mode predict_file --pretrained_model NISQA/weights/nisqa.tar --deg /content/drive/MyDrive/DSP/enhanced/gt_recorded.wav --output_dir ./results

Device: cpu
Model architecture: NISQA_DIM
Loaded pretrained model from NISQA/weights/nisqa.tar
---> Predicting ...
/usr/local/lib/python3.10/dist-packages/librosa/feature/spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)
            deg  mos_pred  noi_pred  dis_pred  col_pred  loud_pred   model
gt_recorded.wav  1.865531  3.780917  4.079943  2.506601   2.042944 NISQAv2


In [17]:
!python NISQA/run_predict.py --mode predict_file --pretrained_model NISQA/weights/nisqa.tar --deg /content/drive/MyDrive/DSP/enhanced/gt_convolved.wav --output_dir ./results

Device: cpu
Model architecture: NISQA_DIM
Loaded pretrained model from NISQA/weights/nisqa.tar
---> Predicting ...
/usr/local/lib/python3.10/dist-packages/librosa/feature/spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)
             deg  mos_pred  noi_pred  dis_pred  col_pred  loud_pred   model
gt_convolved.wav  2.101765  2.968494  3.991333   2.08007   2.328538 NISQAv2


| файл | SNR | SDR | SI-SDR |	PESQ | NISQA (все пять значений) | DNSMOS | MOS |
| --- | --- | --- | --- | --- | --- | --- | --- |
| enhanced/gt_recorded.wav (записанный) | -0.20 | -3.2 | -11.0 | 2.16 |  (1.9, 3.8, 4.1, 2.5, 2.0) | --- | 2 (poor) |
| enhanced/gt_convolved.wav (свернутый) | -0.24 | -22.0 | -53.25 | 4.01 | (2.1, 3.0, 4.0, 2.1, 2.3) | --- | 1 (bad) |

# DNSMOS

Тут нужно было запрашивать какой-то ключ